In [1]:
# let's first install the selenium library
! pip install selenium

## Assignment 3

In [2]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

### Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
# connect to webdriver
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)

# to get url
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : guitars


In [4]:
# to Locate searc_bar by id
search_bar = driver.find_element(By.ID,"twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click() 

### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')#collecting urls of all the guitar
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]') 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)
    
    
    
urls

['https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzgwNzk1MzI2MzQ3ODE2OjE2Njg5NDgzNjY6c3BfYXRmOjIwMDU3MTkzMDU2MjA0OjowOjo&url=%2FKadence-Slowhand-Premium-Acoustic-Instrument%2Fdp%2FB077SZ667X%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2WI89EO3FVJGQ%26keywords%3Dguitars%26qid%3D1668948366%26qu%3DeyJxc2MiOiI2LjAyIiwicXNhIjoiNS42MSIsInFzcCI6IjMuODUifQ%253D%253D%26sprefix%3D%252Caps%252C742%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzgwNzk1MzI2MzQ3ODE2OjE2Njg5NDgzNjY6c3BfYXRmOjIwMDkyOTU1MjkwMTk4OjowOjo&url=%2FGibzs-38-C-Acoustic-Guitar-Black%2Fdp%2FB07LH8P4XK%2Fref%3Dsr_1_2_sspa%3Fcrid%3D2WI89EO3FVJGQ%26keywords%3Dguitars%26qid%3D1668948366%26qu%3DeyJxc2MiOiI2LjAyIiwicXNhIjoiNS42MSIsInFzcCI6IjMuODUifQ%253D%253D%26sprefix%3D%252Caps%252C742%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxMzgwNzk1MzI2MzQ3ODE2OjE2Njg5NDgzNjY6c3BfYXRmOjIwMDkyNjI0MDIxNDk4OjowOjo&url=%2F

In [6]:
Product_name = []
Brand_name= []
Price = []
Return = []
Expected_Delivery = []
Availability = []


#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping product name
    try:
        prod=driver.find_element(By.XPATH,'//span[@class="a-size-base-plus a-color-base a-text-normal"]')
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
            #Scraping brand name
    try:
        brand=driver.find_element(By.XPATH,'//div[@id="bylineInfo_feature_div"]/div/a')
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
            #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element(By.XPATH,'//span[@class="a-color-base a-text-bold"]')
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element(By.XPATH,'//div[@id="availability"]')
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")


In [7]:
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Urls'] = urls

Guitar

,Name,Brand,Price,Return,Expected_Delivery,Availability,Urls
0,-,Visit the Kadence Store,,7 days Replacement,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,-,Brand: Gibz's,"4,498",7 days Returnable,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,-,Visit the VAULT Store,"6,930",7 days Returnable,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,-,Visit the VAULT Store,"7,290",7 days Replacement,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,-,Visit the JUAREZ Store,"2,399",7 days Replacement,-,-,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
153,-,Visit the GIVSUN Store,"4,169",7 days Replacement,-,-,https://www.amazon.in/Diamond-Special-6-String...
154,-,Brand: Martin Smith,"5,825",7 days Replacement,-,Only 1 left in stock,https://www.amazon.in/Martin-Smith-W-101-WHT-P...
155,-,Visit the YAMAHA Store,"11,490",7 days Replacement,-,Only 1 left in stock,https://www.amazon.in/Yamaha-Acoustic-Guitar-D...
156,-,Brand: Fender,"21,999",7 days Replacement,-,Only 1 left in stock,https://www.amazon.in/Fender-FA135CE-Electric-...


In [8]:
driver.close()

### Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [10]:
# connect to webdriver
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)
# Getting the webpage of mentioned url 
url = "https://images.google.com/"
#Creating empty list 
urls = []    #empty list
data = []    #empty list
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.TAG_NAME,"input") 
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    # Clicking on search button
    search_button = driver.find_element(By.XPATH,'//button[@class="Tg7LZd"]').click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(50):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])
    file = open(r"C:\Users\HP\Downloads\google image"+str(i)+".jpg","wb")
    file.write(response.content)

In [11]:
driver.close()

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [25]:
# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
# To get the web page 
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

In [26]:

# to close login pop up
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click()


In [27]:
# to enter the item name which user wants to search
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)


 Enter the name of Smartphone that has to be searched : Oneplus Nord


In [28]:
# To get  urls of phones on 1st page
page1_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [29]:
len(page1_urls)

24

In [30]:
## Details to be scraped: “Brand Name”, “Smartphone name”,
## “Colour”, “RAM”, “Storage(ROM)”,
##“Primary Camera”, “Secondary Camera”,
## “Display Size”, “Battery Capacity”, 
##“Price”, “Product URL”.
# to make empty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [31]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//li[@class="_21lJbe"]')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
      #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
     

    
    #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
     #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')
        
        
    

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-blue-tide-128-gb/p/itm7acae55b999e6?pid=MOBGDWFJFBKYTEGP&lid=LSTMOBGDWFJFBKYTEGPGHG2AM&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=b68c99e2-8103-42a4-a7ca-171a7d0285d7.MOBGDWFJFBKYTEGP.SEARCH&ppt=hp&ppn=homepage&ssid=z2os0t18i80000001668956679481&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGJMUHBTHF8HPM&lid=LSTMOBGJMUHBTHF8HPM53F99J&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=b68c99e2-8103-42a4-a7ca-171a7d0285d7.MOBGJMUHBTHF8HPM.SEARCH&ppt=hp&ppn=homepage&ssid=z2os0t18i80000001668956679481&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGHH9FVUHCFHTY&lid=LSTMOBGHH9FVUHCFHTY6YQ0XR&marketplace=FLIPKART&q=Oneplus+Nord&store=

In [32]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))

24 24 24 24 24


In [33]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,OnePlus,"Handset,SIM Tray Ejector, Adapter, Phone Case,...",Blue Tide,6 GB,128 GB,64MP Rear Camera,16MP Front Camera,16.74 cm (6.59 inch),5000 mAh,"₹19,899",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,975",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,998",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,839",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,610",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,8 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹21,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,749",https://www.flipkart.com/oneplus-nord-ce-2-lit...
7,OnePlus,"Mobile, Adapter, Phone Case, Sim Tray Ejector,...",Gray Shadow,12 GB,256 GB,Primary & Secondary Camera,-,16.94 cm (6.67 inch),-,"₹33,998",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
8,-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/oneplus-nord-ce-2-5g-...
9,OnePlus,"1 Phone, 1 Charger, 1 USB data cable, 1 SIM ej...",Celestial Black,4 GB,64 GB,Primary Camera,-,16.66 cm (6.56 inch),-,"₹14,799",https://www.flipkart.com/oneplus-nord-n20-se-c...


In [34]:
driver.close()

### Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.
### to get web browser

In [35]:

# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
# To get the web page 
# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Ranchi
URL Extracted:  https://www.google.co.in/maps/place/Ranchi,+Jharkhand/@20.9880135,82.7525294,5z/data=!4m5!3m4!1s0x39f4e104aa5db7dd:0xdc09d49d6899f43e!8m2!3d23.3440997!4d85.309562
Latitude = 20.9880135, Longitude = 82.7525294


In [36]:
driver.close()

## Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [37]:
# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)

#to get url
driver.get('https://trak.in/')

#to get xpath for funding deals
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [38]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
      # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df


,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [39]:
driver.close()

### Q7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [2]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

In [14]:
# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)

In [15]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [16]:
#searching for best laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [17]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [18]:
# Scraping the data of laptop names
laptop_name= driver.find_elements(By.XPATH,"//div[@class='TopNumbeHeading']")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass

#Scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [19]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))

10 10 10 10 10 10 10 10 10


In [20]:
data=list(zip(Laptop_Name, Operating_sys, Display, Processor, Memory, Weight, Dimensions, Graph_proc, Price))

In [21]:
df=pd.DataFrame(data,columns=['Laptop_Name','Operating_System','Display','Processor','Memory','Weight','Dimension','Graph_Proc','Price'])

In [22]:
df

,Laptop_Name,Operating_System,Display,Processor,Memory,Weight,Dimension,Graph_Proc,Price
0,1.\nMSI Titan GT77-12UHS,Windows 11 Home OS,"17.3"" (3840 x 2160) Display",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,2.\nAlienware X17 R2,Windows 11 Home OS,"17.3"" (1920 x 1080) Display",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Rs 389,990"
2,3.\nAcer Predator Triton 500 SE PT516-52s,Windows 11 OS,"16"" (2560 x 1600) Display",Windows 11,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,262 x 359 x 20 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,"Rs 229,990"
3,4.\nOmen By HP (16-B1371TX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 176,290"
4,5.\nAcer Predator Helios 300 AN515-45,Windows 11 Home OS,"15.6"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 172,999"
5,6.\nMSI Delta 15 (A5EFK-083IN),Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs 188,990"
6,7.\nOmen By HP (16-C0141AX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs 124,990"
7,8.\nLenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home OS,"16"" (2560 x 1600) Display",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 162,869"
8,9.\nAlienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 174,000"
9,10.\nLenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home OS,15.6 MP | NA Display,Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs 131,990"


In [23]:
driver.close()

### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [46]:
# connect to wed browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth=200
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
import time
pd.options.display.max_colwidth =100
import re
from bs4 import BeautifulSoup
import requests

In [47]:
# opening the url digit.in
url = "https://www.forbes.com/"
driver.get(url)
time.sleep(2)

In [48]:
rank=[]
name=[]
net_worth=[]
age=[]
country=[]
source=[]

In [49]:
try:
    driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
    driver.maximize_window()
    driver.get(url)
    delay=10
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="row-7"]/div[1]/a')))
    #locating the  tag on the page and clicking it
    but=driver.find_element(By.XPATH,'//*[@id="row-7"]/div[1]/a')
    but.click()
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="row-1"]/div/div/div[1]/div/div[1]/div[1]/div[2]/a')))
    #locating the billiniore tag and clicking it
    billi=driver.find_element(By.XPATH,'//*[@id="row-1"]/div/div/div[1]/div/div[1]/div[1]/div[2]/a')
    billi.click()
except TimeoutException as e:
        print("EXCEPTION RAISED ",e)

In [50]:
try:
    #ITERATING OVER ALL THE 14 PAGES USING FOR LOOP
    for i in range(14):
        #LOCATING EACH ROW OF THE TABLE AND BY USING STRING FORMATTING WE WILL APPEND RESPECTIVE LIST
        bil=driver.find_elements(By.XPATH,"//div[@class='table-row-group__container']//div[@role='row']")
        for i in bil:
            kak=i.text.split('\n')
            rank.append(kak[0])
            name.append(kak[1])
            net_worth.append(kak[2])
            age.append(kak[3])
            country.append(kak[4])
            source.append(kak[5])
        #LOCATING THE NEXT BUTTON AND CLICKING IT
        nxt=driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        nxt.click()
        time.sleep(5)
except NoSuchElementException as e:
        print("SCRAPING COMPLETE")
        driver.quit()
        

SCRAPING COMPLETE


In [51]:
#A DATAFRAME ss formed containg all the above lists as columns
ss=pd.DataFrame({'RANK':rank,'NAME':name,'NET_WORTH':net_worth,'COUNTRY':country,'SOURCE':source})

In [52]:
ss

,RANK,NAME,NET_WORTH,COUNTRY,SOURCE
0,1.,Elon Musk,$219 B,United States,"Tesla, SpaceX"
1,2.,Jeff Bezos,$171 B,United States,Amazon
2,3.,Bernard Arnault & family,$158 B,France,LVMH
3,4.,Bill Gates,$129 B,United States,Microsoft
4,5.,Warren Buffett,$118 B,United States,Berkshire Hathaway
...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,China,Fiberglass
2664,2578.,Zhou Ruxin,$1 B,China,Navigation
2665,2578.,Wen Zhou & family,$1 B,China,chemicals
2666,2578.,Zhou Yifeng & family,$1 B,China,liquefied petroleum gas


### Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [53]:
# to get web browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe")
time.sleep(2)
# to get url
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [54]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : Drishyam 2


In [55]:
#clicking on search button
search_btn = driver.find_element(By.ID,"search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [56]:
#clicking on first video
link_click = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

In [57]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements(By.ID,"content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

['Confession hoga, Case Reopen hote hi\n18th November Ko.... ',
 'Best thing about Drishyam 2 is that they kept the old caste intact. Not a single change so that audience can stay connected with them throughout the movie. And when they had to bring a new character inn... they choose one of the best ever for that, AKSHAY KHANNA.',
 "The best thing about this trailer is that it gives the exact same vibe as its prequel. Same cast, same vibes\nCan't believe we're getting a Bollywood movie like this after such a long time ️",
 "Akshay Khanna always nails it in a serious role can't wait to see ️",
 'Kon kon Drishyam 2 ke liye eagerly waiting ️',
 'The whole Bollywood should thanks to Malayalam industry for the this epic master class ',
 "I just watched Drishyam 2 and it's absolutely insane. The climax ",
 'Drishyam is not just a movie, it is a brand and it is a feeling, a emotion ️',
 'This is what we want from Bollywood ',
 'When solid actors play solid characters,we get solid films like Dr

In [58]:
len(comments)

1458

In [60]:
# to scrape time when comment was posted
tm_tags = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500]    

['',
 '4K',
 '',
 '8.6K',
 '',
 '2.4K',
 '',
 '927',
 '',
 '714',
 '',
 '488',
 '',
 '161',
 '',
 '2.4K',
 '',
 '744',
 '',
 '260',
 '',
 '93',
 '',
 '311',
 '',
 '4',
 '',
 '1.1K',
 '',
 '282',
 '',
 '75',
 '',
 '3',
 '',
 '49',
 '',
 '36',
 '',
 '1.8K',
 '',
 '53',
 '',
 '27',
 '',
 '',
 '',
 '65',
 '',
 '1',
 '',
 '1.8K',
 '',
 '2',
 '',
 '9',
 '',
 '39',
 '',
 '1',
 '',
 '2.9K',
 '',
 '35',
 '',
 '37',
 '',
 '1',
 '',
 '21',
 '',
 '3',
 '',
 '871',
 '',
 '19',
 '',
 '4',
 '',
 '15',
 '',
 '1',
 '',
 '18',
 '',
 '726',
 '',
 '3',
 '',
 '1',
 '',
 '6',
 '',
 '29',
 '',
 '8',
 '',
 '1.6K',
 '',
 '16',
 '',
 '1',
 '',
 '24',
 '',
 '4',
 '',
 '1',
 '',
 '636',
 '',
 '53',
 '',
 '1',
 '',
 '6',
 '',
 '7',
 '',
 '2',
 '',
 '6K',
 '',
 '1',
 '',
 '28',
 '',
 '1',
 '',
 '',
 '',
 '1',
 '',
 '345',
 '',
 '',
 '',
 '11',
 '',
 '3',
 '',
 '8',
 '',
 '1',
 '',
 '309',
 '',
 '8',
 '',
 '1',
 '',
 '1',
 '',
 '',
 '',
 '',
 '',
 '364',
 '',
 '5',
 '',
 '',
 '',
 '4',
 '',
 '',
 '',
 '',
 '',
 '',


In [61]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = comment_time[:500]
Youtube['Comment upvotes'] = No_of_Likes[:500]

#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,"Confession hoga, Case Reopen hote hi\n18th November Ko....",1 month ago,4K
1,Best thing about Drishyam 2 is that they kept the old caste intact. Not a single change so that ...,1 month ago,8.6K
2,The best thing about this trailer is that it gives the exact same vibe as its prequel. Same cast...,2 weeks ago,2.4K
3,Akshay Khanna always nails it in a serious role can't wait to see ️,4 days ago,927
4,Kon kon Drishyam 2 ke liye eagerly waiting ️,1 month ago,714
...,...,...,...
495,Best casting,4 days ago,
496,Loved it,15 hours ago,
497,Today i watched this movie in theatre in inorbit mall at 2.00 clock show and i want to say this ...,2 weeks ago,1
498,CANT WAIT,2 days ago,


In [63]:
driver.close()

### Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [64]:
# to get web browser
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe") 
time.sleep(3)

# to get url
url = "https://www.hostelworld.com/"
driver.get(url)

In [70]:
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe") 
driver.maximize_window()
driver.get(url)
time.sleep(5)
#LOCATING THE SEARCH BAR AND CLICKING ON IT
ee=driver.find_element(By.XPATH,"//div[@class='field-inner']")
ee.click()
time.sleep(5)
#SENDING "LONDON " AS THE KEYS IN THE SEARCH BAR
hs=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input")
time.sleep(3)
hs.send_keys('London ')
time.sleep(10)
#FROM THE SUGGESTED POPDOWN SELECTING THE SUITABLE OPTION AND CLICKING ON IT
sel=driver.find_element(By.XPATH,"//ul[@id='predicted-search-results']//li[2]")
sel.click()
#LOCATING THE GO BUTTON AND CLICKING ON IT
bu=driver.find_element(By.XPATH,"//div[@class='search-button']//button")
bu.click()

In [71]:
#CREATING A LIST WEE
#LOCATING ALL THE URLS OF THE HOSTELS PRESENT ON THAT PAGE AND APPENDING IT IN WEE
wee=[]
fr=driver.find_elements(By.XPATH,"//h2[@class='title title-6']//a")
for i in fr:
    wee.append(i.get_attribute('href'))

In [74]:
#ITERATING IN THE WEE LIST AND GETTING ALL THE URLS
for i in wee:
    driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver.exe") 
    driver.maximize_window()
    driver.get(i)
    time.sleep(5)
    print('\n\n\n\n-**************************************************************************************************************-\n\n\n')
    #SCRAPING ALL THE NECESSARY INFORMATION FROM THE PAGE USING DIFFERENT ELEMENTS AND PRINTING THEM
    naa=driver.find_element(By.XPATH,"//div[@class='title-2']")
    print(f"\nNAME OF THE HOSTEL  : {naa.text}")
    try:
        rr=driver.find_element(By.XPATH,"//div[@class='score orange big']")
        print(f"\nRATING OF THE HOSTEL : {rr.text}")
    except NoSuchElementException as e:
        rr=driver.find_element(By.XPATH,"//div[@class='score gray big']")
        print(f"\nRATING OF THE HOSTEL : {rr.text}")
    ove=driver.find_element(By.XPATH,"//div[@class='keyword']")
    print(f"\nOVER ALL REVIEW OF THE HOSTEL : {ove.text}")
    totr=driver.find_element(By.XPATH,"//div[@class='reviews']")
    print(f"\nTOTAL REVIEWS OF THE HOSTEL : {totr.text}")
    fac=driver.find_elements(By.XPATH,"//div[@class='rating-factors rating-factors big']//div")
    print('\n\n------FEATURES----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n')
    for i in fac:
        print(i.text)
    pd=driver.find_element(By.XPATH,"//div[@class='content collapse-content']")
    print('\n\nPROPERTY DESCRIPTION :\n\n')
    print(pd.text)
    ff=driver.find_elements(By.XPATH,"//ul[@class='facility-sections']//li")
    print('\n\nFACILITIES----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n:')
    for i in ff:
        print('\n\n')
        print(i.text)
    print('\n\nTYPE AND PRICES-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n\n')    
    char=driver.find_elements(By.XPATH,"//div[@class='room-container desktop']")
    for i in char:
        print('\n\n************************************************************************\n\n')
        print(i.text)
    driver.quit()
    





-**************************************************************************************************************-




NAME OF THE HOSTEL  : Wombat's City Hostel London

RATING OF THE HOSTEL : 8.9

OVER ALL REVIEW OF THE HOSTEL : Fabulous

TOTAL REVIEWS OF THE HOSTEL : 14326 Total Reviews


------FEATURES----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Perfect Location
Superb Staff
Fantastic Cleanliness


PROPERTY DESCRIPTION :


A safe haven in the middle of the metropolis: We took a traditional brick building with roots going back to the 1800s and set a new standard for hostel life in the UK.
What used to be sailors' lodgings in the olden days is now a hostel satisfying all needs of the modern traveler. With great attention to detail, we have created a hostel in London that celebrates the legacy of our historic building.
Old oak that 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='score gray big']"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x011EACD3+2075859]
	Ordinal0 [0x0117EE61+1633889]
	Ordinal0 [0x0107B7BD+571325]
	Ordinal0 [0x010AAC2F+764975]
	Ordinal0 [0x010AAE1B+765467]
	Ordinal0 [0x010DD0F2+970994]
	Ordinal0 [0x010C7364+881508]
	Ordinal0 [0x010DB56A+963946]
	Ordinal0 [0x010C7136+880950]
	Ordinal0 [0x0109FEFD+720637]
	Ordinal0 [0x010A0F3F+724799]
	GetHandleVerifier [0x0149EED2+2769538]
	GetHandleVerifier [0x01490D95+2711877]
	GetHandleVerifier [0x0127A03A+521194]
	GetHandleVerifier [0x01278DA0+516432]
	Ordinal0 [0x0118682C+1665068]
	Ordinal0 [0x0118B128+1683752]
	Ordinal0 [0x0118B215+1683989]
	Ordinal0 [0x01196484+1729668]
	BaseThreadInitThunk [0x75ADFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77A97BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A97B8E+238]


## since no rating are found in Beaconsfield so the error appears 